This notebook implements basic RAG functionality. The notebook tries to get the answer for the question "Who won the last cricket world cup and in which year?" from the LLM 'gpt-3.5-turbo'. Since the model was trained on 2021 data, it responds with an outdated answer. Then it pulls the contents of page https://en.wikipedia.org/wiki/2023_Cricket_World_Cup and stores the same in the knowledgebase. Then it uses RAG to answer the same question and gets the correct answer.

The notebook makes use of OpenAI APIs, LangChain and Chroma DB.

Please use your OpenAI_API key in cell 1.

In [ ]:
#set the OPENAI_API_KEY environment variable
import os
os.environ['OPENAI_API_KEY']="<ADD YOUR KEY HERE>"

In [2]:
#use langchain webloader to load the page https://en.wikipedia.org/wiki/2023_Cricket_World_Cup

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/2023_Cricket_World_Cup",),
)

docs=loader.load()
#docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
#split the docs
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)
# split_docs

In [4]:
#get the embedding
from langchain_openai.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings()


#store in vector store
from langchain_chroma import Chroma
db = Chroma.from_documents(split_docs, embedding)

#create a retriever
retriever = db.as_retriever()

In [5]:
#create a prompt template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
                                          Answer the question based on the context:
                                          {context}
                                          
                                          Question: {question}""")

In [6]:
#create the model
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
question = 'Who won the last cricker world cup and in which year?"?'

In [8]:
#call the model without RAG
response = model.invoke(question)
response.content

'The last Cricket World Cup was won by England in 2019.'

As you can see the LLM is giving an outdated response

In [11]:
#create a chain
chain = prompt | model

In [12]:
#get the context details
retrieved_content = retriever.invoke(question)

In [13]:
#invoke using RAG with the same question
response = chain.invoke({'context':retrieved_content,
              'question':question})
response.content

'Australia won the last Cricket World Cup in 2023.'